Clone the respository

In [2]:
!git clone https://github.com/Zerve-AI/pypelines.git

Installing the pypeline

In [2]:
import os
folder = 'C:/zerve'
os.chdir(f'{folder}/pypelines')

In [3]:
!pip install .

Processing c:\zerve\pypelines
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'


ERROR: Could not find a version that satisfies the requirement esig==0.9.8.3sktime (from pypelines) (from versions: 0.0.0, 0.6.6, 0.6.7, 0.6.8, 0.6.9, 0.6.10, 0.6.11, 0.6.12, 0.6.13, 0.6.22, 0.6.31, 0.7.1, 0.7.3, 0.7.4, 0.8.0, 0.8.1, 0.8.2, 0.9.0, 0.9.1, 0.9.4, 0.9.5, 0.9.6, 0.9.7, 0.9.8, 0.9.8.1, 0.9.8.2, 0.9.8.3)
ERROR: No matching distribution found for esig==0.9.8.3sktime


LIST OF DATAPREP METHODS

In [4]:
from pypelines import utils

utils.list_supported_dataprepmethods(method_type='forecasting')

['LagFeatures', 'DatetimeFeatures', 'DatetimeSubtraction']

Loading Library

In [5]:
import pypelines.data_prep_pipeline as pipe
import pandas as pd

Data Load and Method Selection

In [7]:
housing = pd.read_csv("pypelines/datasets/regression/housing.csv")
dataprep_pypelines_all = pipe.DataPrepPipeline(
    data = housing, 
    target = 'population',
    preprocessing_method = 'MatchVariables',
    outlier_method = 'Winsorizer', 
    numerical_imputation_method = 'MeanMedianImputer', 
    categorical_imputation_method = 'CategoricalImputer', 
    encoding_method = 'RareLabelEncoder',
    datetime_method = None,
    target_date1_column = None,
    target_date2_column = None,
    discretisation_method = 'EqualFrequencyDiscretiser',
    forecasting_method='LagFeatures')

Data Preparation code generation

In [8]:
dataprep_pypelines_all.code_to_clipboard()

In [9]:

import numpy as np
import pandas as pd
from feature_engine.preprocessing import MatchVariables
from feature_engine.outliers import Winsorizer
from feature_engine.discretisation import EqualFrequencyDiscretiser
from feature_engine.imputation import MeanMedianImputer
from feature_engine.imputation import CategoricalImputer

from feature_engine.encoding import RareLabelEncoder


encode = RareLabelEncoder(tol=0.05, n_categories=10, max_n_categories=None, replace_with='Rare', variables=None, missing_values='raise', ignore_format=False)
from feature_engine.timeseries.forecasting import LagFeatures

# target dataframe: housing
target = "population"
features = list(housing.columns.drop("population"))
feature_df = housing[features]




# Preprocessing
try:
    
    process = MatchVariables(missing_values='raise', verbose=True)
    housing = process.fit_transform(housing)

    
except Exception as e:
    print("Error in outlier:", str(e))




# Handling Outliers
try:
    
    out = Winsorizer(capping_method='gaussian', tail='right', fold=3, add_indicators=False, variables=None, missing_values='ignore')
    housing = out.fit_transform(housing)
    
except Exception as e:
    print("Error in outlier:", str(e))
 


# Missing Value Imputation

# Identifying the missing columns and selecting only columns with less than 10% missing values
edited_missing_columns = feature_df.columns[feature_df.isnull().mean() <= 0.1].tolist()

if len(edited_missing_columns) != 0:
    int_lst = housing[edited_missing_columns].select_dtypes(include=['int64', 'float64']).columns.tolist()
    cat_lst = housing[edited_missing_columns].select_dtypes(include=['category','object']).columns.tolist()
    try:
        if len(int_lst) > 0:
            
            imputer = MeanMedianImputer(imputation_method='median', variables=int_lst)
            housing_num = imputer.fit_transform(housing[int_lst])

            
    except Exception as e:
        print("Error in integer imputation:", str(e))

    try:
        if len(cat_lst) > 0:
            
            imputer = CategoricalImputer(imputation_method='missing',variables=cat_lst)
            housing_cat = imputer.fit_transform(housing[cat_lst])

            
    except Exception as e:
        print("Error in integer imputation:", str(e))
        
feature_df = pd.concat([housing_num,housing_cat],axis=1)
housing =  pd.concat([feature_df,housing["population"]],axis=1)




# Encoding Categorical Variables
try:
    
    encode = RareLabelEncoder(tol=0.05, n_categories=10, max_n_categories=None, replace_with='Rare', variables=None, missing_values='raise', ignore_format=False)
    housing = encode.fit_transform(housing)
    
except Exception as e:
    print("Error in integer encoding:", str(e))





# Discretisation
try:
    
    discret = EqualFrequencyDiscretiser(variables=None, q=10, return_object=False, return_boundaries=False, precision=3)
    x = housing.drop("population", axis=1)
    x = discret.fit_transform(x)

    
except Exception as e:
    print("Error in integer encoding:", str(e))



# Forecasting Features
try:
    
    lf = LagFeatures(variables=None, periods=[1,2], freq=None, sort_index=True, missing_values='raise', drop_original=False)
    x = lf.fit_transform(housing)


    
except Exception as e:
    print("Error in integer forecast:", str(e))

##### End of Data Processing Pipeline #####



Error in outlier: Some of the variables in the dataset contain NaN. Check and remove those before using this transformer.


c:\Users\umerk\anaconda3\envs\canvas\lib\site-packages\feature_engine\encoding\rare_label.py:216: UserWarning: The number of unique categories for variable ocean_proximity is less than that indicated in n_categories. Thus, all categories will be considered frequent
  warnings.warn(
